In [58]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from keras.utils.np_utils import to_categorical
from keras.models import Sequential
from keras.layers import ConvLSTM2D, Activation
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPool2D, BatchNormalization
from keras.optimizers import Adam
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import LearningRateScheduler

## Useful functions

In [59]:
def save_model(model, filename):
    model_json = model.to_json()
    with open(filename + ".json", "w") as json_file:
        json_file.write(model_json)
    # serialize weights to HDF5
    model.save_weights(filename + ".h5")
    print("Saved model to disk")

## Load data

In [60]:
DATA_FOLDER = "./data/"

In [61]:
df_training = pd.read_parquet(DATA_FOLDER + "training.parquet")

In [62]:
df_public_test = pd.read_parquet(DATA_FOLDER + "public_test.parquet")

In [63]:
df_private_test = pd.read_parquet(DATA_FOLDER + "private_test.parquet")

## Process data

In [64]:
x_train = df_training.as_matrix()[:,1:]
y_train = df_training.as_matrix()[:,0]
x_test = df_public_test.as_matrix()[:,1:]
y_test = df_public_test.as_matrix()[:,0]

c:\python3\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  """Entry point for launching an IPython kernel.
c:\python3\lib\site-packages\ipykernel_launcher.py:2: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  
c:\python3\lib\site-packages\ipykernel_launcher.py:3: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  This is separate from the ipykernel package so we can avoid doing imports until
c:\python3\lib\site-packages\ipykernel_launcher.py:4: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  after removing the cwd from sys.path.


In [65]:
x_train = x_train.reshape(-1, 48, 48, 1)
x_test = x_test.reshape(-1, 48, 48, 1)

In [66]:
x_train = x_train.astype("float32")/255.
x_test = x_test.astype("float32")/255.

In [67]:
y_train = to_categorical(y_train)
y_test = to_categorical(y_test)

In [68]:
x_train.shape

(28709, 48, 48, 1)

## Create model

In [69]:
model = Sequential()

model.add(Conv2D(64, 3, data_format="channels_last", kernel_initializer="he_normal", 
                 input_shape=(48, 48, 1)))
model.add(BatchNormalization())
model.add(Activation("relu"))

model.add(Conv2D(64, 3))
model.add(BatchNormalization())
model.add(Activation("relu"))
model.add(MaxPool2D(pool_size=(2, 2), strides=2))
model.add(Dropout(0.6))

model.add(Conv2D(32, 3))
model.add(BatchNormalization())
model.add(Activation("relu"))

model.add(Conv2D(32, 3))
model.add(BatchNormalization())
model.add(Activation("relu"))

model.add(Conv2D(32, 3))
model.add(BatchNormalization())
model.add(Activation("relu"))
model.add(MaxPool2D(pool_size=(2, 2), strides=2))
model.add(Dropout(0.6))

model.add(Flatten())
model.add(Dense(128))
model.add(BatchNormalization())
model.add(Activation("relu"))
model.add(Dropout(0.6))

model.add(Dense(7))
model.add(Activation('softmax'))

In [70]:
from keras.utils import plot_model
plot_model(model, to_file='model.png', show_shapes=True, show_layer_names=False)

In [71]:
datagen = ImageDataGenerator(zoom_range = 0.1,
                            height_shift_range = 0.1,
                            width_shift_range = 0.1,
                            rotation_range = 10)

In [72]:
model.compile(loss='categorical_crossentropy', optimizer = Adam(lr=1e-4), metrics=["accuracy"])

In [73]:
annealer = LearningRateScheduler(lambda x: 1e-3 * 0.9 ** x)

In [74]:
hist = model.fit_generator(datagen.flow(x_train, y_train, batch_size=16),
                           steps_per_epoch=500,
                           epochs=20, #Increase this when not on Kaggle kernel
                           validation_data=(x_test[:400,:], y_test[:400,:]), #For speed
                           callbacks=[annealer])

Epoch 1/20
500/500 [==============================] - 675s 1s/step - loss: 2.1542 - acc: 0.2044 - val_loss: 1.7786 - val_acc: 0.3175
Epoch 2/20
500/500 [==============================] - 684s 1s/step - loss: 1.8688 - acc: 0.2461 - val_loss: 1.7286 - val_acc: 0.3225
Epoch 3/20
500/500 [==============================] - 752s 2s/step - loss: 1.7953 - acc: 0.2700 - val_loss: 1.7102 - val_acc: 0.2950
Epoch 4/20
500/500 [==============================] - 794s 2s/step - loss: 1.7686 - acc: 0.2780 - val_loss: 1.6968 - val_acc: 0.3425
Epoch 5/20
500/500 [==============================] - 703s 1s/step - loss: 1.7384 - acc: 0.2984 - val_loss: 1.6621 - val_acc: 0.3225
Epoch 6/20
500/500 [==============================] - 683s 1s/step - loss: 1.7008 - acc: 0.3138 - val_loss: 1.6361 - val_acc: 0.3675
Epoch 7/20
500/500 [==============================] - 691s 1s/step - loss: 1.6955 - acc: 0.3181 - val_loss: 1.5994 - val_acc: 0.3700
Epoch 8/20
500/500 [==============================] - 628s 1s/step - 

In [18]:
hist = model.fit_generator(datagen.flow(x_train, y_train, batch_size=16),
                           steps_per_epoch=500,
                           initial_epoch=20,
                           epochs=40, #Increase this when not on Kaggle kernel
                           validation_data=(x_test[:400,:], y_test[:400,:]), #For speed
                           callbacks=[annealer])

Epoch 21/40
500/500 [==============================] - 516s 1s/step - loss: 1.3772 - acc: 0.4612 - val_loss: 1.2512 - val_acc: 0.5450
Epoch 22/40
500/500 [==============================] - 520s 1s/step - loss: 1.3944 - acc: 0.4564 - val_loss: 1.2297 - val_acc: 0.5475
Epoch 23/40
500/500 [==============================] - 521s 1s/step - loss: 1.3660 - acc: 0.4641 - val_loss: 1.2258 - val_acc: 0.5600
Epoch 24/40
500/500 [==============================] - 520s 1s/step - loss: 1.3788 - acc: 0.4586 - val_loss: 1.2210 - val_acc: 0.5400
Epoch 25/40
500/500 [==============================] - 520s 1s/step - loss: 1.3569 - acc: 0.4647 - val_loss: 1.2043 - val_acc: 0.5500
Epoch 26/40
500/500 [==============================] - 520s 1s/step - loss: 1.3376 - acc: 0.4768 - val_loss: 1.1927 - val_acc: 0.5500
Epoch 27/40
500/500 [==============================] - 519s 1s/step - loss: 1.3527 - acc: 0.4777 - val_loss: 1.1985 - val_acc: 0.5525
Epoch 28/40
500/500 [==============================] - 519s 1s

In [ ]:
final_loss, final_acc = model.evaluate(x_test, y_test, verbose=0)
print("Final loss: {0:.4f}, final accuracy: {1:.4f}".format(final_loss, final_acc))

In [75]:
save_model(model, "./trained_models/fifth_model")

Saved model to disk


In [ ]:
def plot_history(history):
    plt.plot(history.history['acc'])
    plt.plot(history.history['val_acc'])
    plt.title('model accuracy')
    plt.ylabel('accuracy')
    plt.xlabel('epoch')
    plt.legend(['train', 'test'], loc='upper left')
    plt.show()
    # summarize history for loss
    plt.plot(history.history['loss'])
    plt.plot(history.history['val_loss'])
    plt.title('model loss')
    plt.ylabel('loss')
    plt.xlabel('epoch')
    plt.legend(['train', 'test'], loc='upper left')
    plt.show()

In [ ]:
plot_history(hist)